<a href="https://colab.research.google.com/github/stefandanielachirei/licenta_2023_Ceica_Sergiu/blob/work/Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/content/kaggle/input/train.csv.zip
/content/kaggle/input/SampleSolution.csv
/content/kaggle/input/test.csv.zip


In [22]:
# read dataset
dataSet = pd.read_csv("/content/kaggle/input/train.csv.zip")

# check data for NaN
print(dataSet.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27674 entries, 0 to 27673
Data columns (total 83 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   patient_id                             27674 non-null  int64  
 1   patient_race                           12522 non-null  object 
 2   payer_type                             24838 non-null  object 
 3   patient_state                          27595 non-null  object 
 4   patient_zip3                           27674 non-null  int64  
 5   patient_age                            27674 non-null  int64  
 6   patient_gender                         27674 non-null  object 
 7   bmi                                    8749 non-null   float64
 8   breast_cancer_diagnosis_code           27674 non-null  object 
 9   breast_cancer_diagnosis_desc           27674 non-null  object 
 10  breast_cancer_diagnosis_year           27674 non-null  int64  
 11  me

In [23]:
# check for the number of missing values
missingValues = dataSet.isnull().sum()
pd.set_option('display.max_rows', 500)
display(missingValues)

patient_id                                   0
patient_race                             15152
payer_type                                2836
patient_state                               79
patient_zip3                                 0
patient_age                                  0
patient_gender                               0
bmi                                      18925
breast_cancer_diagnosis_code                 0
breast_cancer_diagnosis_desc                 0
breast_cancer_diagnosis_year                 0
metastatic_cancer_diagnosis_code             0
metastatic_first_treatment                   0
metastatic_first_treatment_type          11024
metastatic_first_novel_treatment         27441
metastatic_first_novel_treatment_type    27441
region                                     223
division                                   223
population                                   0
density                                      0
age_median                                   0
age_under_10 

In [24]:
# Get general description of data
dataSet.describe()

,patient_id,patient_zip3,patient_age,bmi,breast_cancer_diagnosis_year,population,density,age_median,age_under_10,age_10_to_19,...,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,treatment_pd
count,27674.000000,27674.000000,27674.000000,8749.000000,27674.000000,27674.000000,27674.000000,27674.000000,27674.000000,27674.000000,...,27674.000000,27674.000000,27674.000000,27674.000000,27671.000000,27671.000000,27674.000000,27674.000000,27674.000000,27674.000000
mean,549586.828612,522.024752,54.456421,29.277804,2016.486449,19919.850799,1655.023291,40.674047,11.117255,12.883595,...,5.352033,6.473921,17.862817,13.432810,13.324371,4.603184,28.061654,8.573778,7.079766,134.218472
std,260306.291181,292.405178,11.202089,5.636923,1.058602,13465.788962,3639.259482,4.026393,1.535440,1.880894,...,5.955250,3.539440,17.513898,3.801799,5.724183,6.595424,4.893231,4.306563,3.098581,187.759654
min,100051.000000,6.000000,19.000000,14.000000,2015.000000,636.000000,0.821739,20.600000,0.000000,6.314286,...,0.002564,0.433333,0.060714,4.600000,3.433333,0.000000,12.460784,1.165385,1.200000,0.000000
25%,323742.250000,294.000000,47.000000,25.000000,2016.000000,9080.000000,169.322727,37.734426,10.149254,11.788406,...,1.307353,3.892593,4.608824,10.270492,9.341176,0.980851,24.968182,5.497143,4.972222,36.000000
50%,549720.000000,490.000000,55.000000,28.820000,2016.000000,18355.000000,627.904167,40.761765,11.022034,12.923944,...,3.355556,5.578689,11.588889,13.069492,12.122857,2.636000,27.814286,7.441772,6.882222,70.000000
75%,775422.250000,784.000000,62.000000,33.000000,2017.000000,27796.000000,1555.107692,43.175862,12.194737,13.930435,...,7.630000,8.663889,24.485185,15.611765,16.408333,5.958333,30.918519,10.685417,8.678846,133.000000
max,999996.000000,999.000000,91.000000,97.000000,2018.000000,71374.000000,29851.685710,57.417647,17.675000,35.300000,...,33.188889,26.429870,99.104348,35.155556,48.931818,74.067391,48.020000,29.565000,25.200000,1455.000000


In [25]:
# Get dataset column names
dataSet.columns

Index(['patient_id', 'patient_race', 'payer_type', 'patient_state',
       'patient_zip3', 'patient_age', 'patient_gender', 'bmi',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'breast_cancer_diagnosis_year', 'metastatic_cancer_diagnosis_code',
       'metastatic_first_treatment', 'metastatic_first_treatment_type',
       'metastatic_first_novel_treatment',
       'metastatic_first_novel_treatment_type', 'region', 'division',
       'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19',
       'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s',
       'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married',
       'widowed', 'family_size', 'family_dual_income',
       'income_household_median', 'income_household_under_5',
       'income_household_5_to_10', 'income_household_10_to_15',
       'income_household_15_to_20', 'income_household_20_to_25',
       'income_household_25_to_35', 'income_household_35_to_5

In [26]:
# Now that we know many details about our dataset, we can go ahead and make sure our data is clean and does not have missing values.
# Some technics for data processing:
    # 1. Remove rows with missing data -> it can reduce the amount of data dramatically and affect your prediction result
    # 2. Impute missing values -> it can be used to impute missing values with meaningful values. Common imputation methods: using mean, mode, median
        # or advanced methods such as regression imputation. In simple words regression imputation uses existing data to predict missing data.

# For now lets simply delete rows with missing data

new_dataset = dataSet.dropna(axis=0)
new_dataset.describe()

,patient_id,patient_zip3,patient_age,bmi,breast_cancer_diagnosis_year,population,density,age_median,age_under_10,age_10_to_19,...,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,treatment_pd
count,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,...,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000
mean,548686.384615,567.615385,50.653846,28.562308,2016.038462,19579.615385,1441.389813,40.938629,11.259203,13.107102,...,4.793791,6.472825,15.852145,12.504549,11.710157,4.698737,27.447651,7.169956,7.278489,353.769231
std,240060.609710,275.868386,12.868387,4.831251,0.999230,14357.104750,3344.704260,3.715439,1.274834,1.743373,...,4.836537,3.194211,14.234054,2.669084,4.694123,5.345701,5.101061,2.805514,2.671996,332.122183
min,118155.000000,112.000000,36.000000,20.120000,2015.000000,3787.000000,7.278261,30.371429,7.691304,10.414894,...,0.567532,2.528571,2.028889,8.845238,5.127027,0.190566,15.407143,4.077778,1.755263,2.000000
25%,372819.250000,364.500000,41.250000,25.505000,2015.000000,10620.000000,208.747856,40.249129,10.328139,12.148452,...,0.993416,4.109382,4.645189,10.213676,8.865162,0.931948,24.353488,4.777273,5.491171,60.500000
50%,520055.500000,600.500000,48.500000,28.975000,2016.000000,16773.000000,577.128320,41.611429,11.413533,13.007120,...,3.797094,5.737692,10.775857,12.448831,10.442250,3.206004,27.779952,6.949091,7.406140,251.500000
75%,722333.750000,809.000000,56.750000,31.727500,2017.000000,22670.750000,1051.832917,42.533522,12.168900,14.076014,...,5.976836,8.508333,20.385246,14.270092,12.589988,5.218813,30.122768,8.083132,9.049245,570.500000
max,935440.000000,928.000000,91.000000,38.560000,2018.000000,71374.000000,17326.407890,50.465217,13.421311,18.864286,...,17.525532,13.112766,48.789362,17.800000,23.346154,20.020000,42.502632,14.700000,11.195349,1134.000000


In [27]:
# Select some features and save them in a list
# There are advanced technics for featutre selection such as Correlation Analysis,Recursive Feature Elimination (RFE), etc.
# For now we simply doing it manualy

important_features = ['patient_age', 'population', 'density']

# create a dataset based on selected features and new data set
X = new_dataset[important_features]
X.describe()

,patient_age,population,density
count,26.000000,26.000000,26.000000
mean,50.653846,19579.615385,1441.389813
std,12.868387,14357.104750,3344.704260
min,36.000000,3787.000000,7.278261
25%,41.250000,10620.000000,208.747856
50%,48.500000,16773.000000,577.128320
75%,56.750000,22670.750000,1051.832917
max,91.000000,71374.000000,17326.407890


In [28]:
X.head()

,patient_age,population,density
137,41,23266,743.557143
451,58,19826,437.300000
2561,51,3787,7.278261
4842,47,18397,927.830000
7965,77,10227,452.832727


In [29]:
# Build simple model using sklearn
# create target
y = new_dataset.treatment_pd
dataset_model = DecisionTreeRegressor(random_state=1)
dataset_model.fit(X,y)

DecisionTreeRegressor(random_state=1)

In [30]:
#simple prediction example (its based on three features)

print(X)
print("The predictions are: ")
print(dataset_model.predict(X))

       patient_age  population       density
137             41       23266    743.557143
451             58       19826    437.300000
2561            51        3787      7.278261
4842            47       18397    927.830000
7965            77       10227    452.832727
8671            53       40163    875.119355
9091            39       12912    292.373684
9499            47       39122   2295.939394
9594            61        8101    246.281013
10040           39       13386    124.465000
10284           60       16996   1614.772059
15770           36       18717   1019.506667
16953           39       12948   1662.475000
17309           57        4435     68.001887
18545           36       16550    245.097917
18762           37       11799    137.880000
18965           56       25744    981.763077
19417           42       16550    245.097917
19969           54       20161   1062.608333
20923           54       32795   1896.220930
21637           91        9521    163.150633
22192     

In [31]:
# lets compare our prediction with real values
print(f"New dataset treatment_pd column: \n{new_dataset.treatment_pd}\n\nPredictions: {dataset_model.predict(X)}")

New dataset treatment_pd column: 
137       786
451        26
2561       23
4842       24
7965      530
8671      129
9091      422
9499      190
9594      743
10040     131
10284     584
15770      60
16953      16
17309     495
18545     186
18762     393
18965     661
19417     316
19969     313
20923     981
21637      36
22192     171
22696       2
23833    1134
26495     784
27265      62
Name: treatment_pd, dtype: int64

Predictions: [ 786.   26.   23.   24.  530.  129.  422.  190.  743.  131.  584.   60.
   16.  495.  186.  393.  661.  316.  313.  981.   36.  171.   32. 1134.
  784.   32.]


In [32]:
# Evaluating the accuracy of the prediction using Mean Absolute Error (MAE)
# This is  "In Sample" score, meaning we used same sample for building and evaluating the model.

mean_absolute_error(y, dataset_model.predict(X))

2.3076923076923075

In [36]:
# Read test data
test_data = pd.read_csv("/content/kaggle/input/test.csv.zip")

X_test = test_data[important_features]
X_test.describe()


,patient_age,population,density
count,11845.000000,11845.000000,11845.000000
mean,54.348417,19813.338117,1660.511727
std,11.252600,13544.525628,3649.073983
min,18.000000,11.000000,0.821739
25%,47.000000,8864.000000,163.150633
50%,55.000000,17969.000000,626.236667
75%,62.000000,27504.000000,1555.107692
max,91.000000,71374.000000,29851.685710


In [37]:
print(dataset_model.predict(X_test))

[661. 316. 530. ... 316. 584. 743.]


In [38]:
train_data_set = dataSet.iloc[:11845]

test_y = train_data_set.treatment_pd
mean_absolute_error(test_y, dataset_model.predict(X_test))


367.9155761924863

In [39]:
from sklearn.ensemble import RandomForestRegressor

ran_forest_model = RandomForestRegressor(random_state=1)

train_y_ran = train_data_set.treatment_pd
X_train_ran = train_data_set[important_features]

ran_forest_model.fit(X_train_ran, train_y_ran)

ran_model_predict = ran_forest_model.predict(X_test)
ran_model_predict

array([ 65.34678571, 110.65533333, 166.98775   , ...,  73.46733333,
       151.545     ,  52.81725   ])

In [40]:
mean_absolute_error(test_y, ran_forest_model.predict(X_test))

130.4891453928685